In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import os
import csv
from tqdm import tqdm

In [ ]:
# с элмо
class Combiner:
    def __init__(self, path, column_name):
        self._path_to_data_dir = path
        self._data_files = sorted(os.listdir(os.path.join(self._path_to_data_dir, 'test')))
        self._data_ml = os.path.join(self._path_to_data_dir,'test_ml', '120_5_3_150_chars_elmo')
        self._data_rules = os.path.join(self._path_to_data_dir, 'rules')
        self.column_name = column_name

    def combine(self):
        # перебираем имена файлов
        for data_file in tqdm(self._data_files, desc='Adding data'):
            model_tags = list()
            rules_tags = list()
            # открываем файл с тегами, полученными моделью
            with open(os.path.join(self._data_ml, data_file), 'r', encoding='utf-8') as data_f:
                reader = csv.DictReader(data_f)
                for row in reader:
                    model_tags.append(row['predicted_ml_tag'])
            with open(os.path.join(self._data_rules, data_file), 'r', encoding='utf-8') as data_f:
                rules_reader = csv.DictReader(data_f)
                # записываем результаты в новый файл
                with open(os.path.join(self._path_to_data_dir, 'combined', data_file), 'w', encoding='utf-8', newline='') as task:
                        fieldnames = ['token', 'tag', self.column_name]
                        writer = csv.DictWriter(task, fieldnames=fieldnames)
                        writer.writeheader()
                        for i, row in enumerate(rules_reader):
                            ml_tag = model_tags[i]     
                            rules_tag = row['predicted_rules_tag']      
                            gtags = ['B-DATE', 'I-DATE', 'B-DURATION', 'I-DURATION'] 
                            mtags = ['B-TIME', 'I-TIME', 'B-SET', 'I-SET']

                            if rules_tag != ml_tag:
                                if (rules_tag.endswith('SET') == True) or (ml_tag.endswith('SET')== True) :
                                    result_tag = rules_tag
                                else:
                                    result_tag = ml_tag
                                
                                writer.writerow({'token' : row['token'], 'tag' : row['tag'], self.column_name: result_tag})  
                            else:
                                writer.writerow({'token' : row['token'], 'tag' : row['tag'], self.column_name: ml_tag})

In [ ]:
combiner = Combiner('/content/gdrive/My Drive/Mezentseva Zavarzina/experiments_dl_model/', 'predicted_combined_tag')
combiner.combine()

Adding data: 100%|██████████| 20/20 [00:00<00:00, 89.98it/s]


In [ ]:
# чистка от того, чтобы начальный тег был начальным

In [ ]:
!pip install seqeval
from seqeval.metrics import classification_report

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=7cbbb27268b854505ac7d3e7cddfb8561939005278d80693f0e3dedff6822f73
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [ ]:
class Evaluator:
    """ Класс для оценки качества извлечения временных выражений """

    def __init__(self, path, column_pred):
        self._predicted_files_dir = path
        self._predicted_files = sorted(os.listdir(self._predicted_files_dir))
        self.column_pred = column_pred

    def evaluate(self) -> str:
        """ Оценка качества извлечения временных выражений
        :return: метрики: precision, recall, F-1 score для каждого класса отдельно и для всех усреднённые
        """
        preds, targets = self._load_predictons()
        report = classification_report(y_true=targets, y_pred=preds)
        return report

    def _load_predictons(self):
        """ Загружает targets и predictions из заранее сформированных csv файлов
        :return: predictions, targets
        """
        preds = []
        targets = []
        for predicted_file in self._predicted_files:
            with open(os.path.join(self._predicted_files_dir, predicted_file), 'r') as predicted_file:
                reader = csv.DictReader(predicted_file)
                preds_from_file = []
                targets_from_file = []
                for row in reader:
                    preds_from_file.append(row[str(self.column_pred)])
                    targets_from_file.append(row['tag'])
                preds.append(preds_from_file)
                targets.append(targets_from_file)
        return preds, targets


In [ ]:
evaluator = Evaluator('/content/gdrive/My Drive/Mezentseva Zavarzina/experiments_dl_model/combined', 'predicted_combined_tag')
report = evaluator.evaluate()
print(report)

           precision    recall  f1-score   support

 DURATION       0.41      0.51      0.46        35
     DATE       0.83      0.68      0.75       101
      SET       1.00      0.25      0.40         4
     TIME       0.50      0.25      0.33         4

micro avg       0.68      0.62      0.65       144
macro avg       0.72      0.62      0.66       144



In [ ]:
# то, как было без элмо
class Combiner:
    def __init__(self, path, column_name):
        self._path_to_data_dir = path
        self._data_files = sorted(os.listdir(os.path.join(self._path_to_data_dir, 'test')))
        self._data_ml = os.path.join(self._path_to_data_dir,'test_ml', '120_5_3_150_chars_elmo')
        self._data_rules = os.path.join(self._path_to_data_dir, 'rules')
        self.column_name = column_name

    def combine(self):
        # перебираем имена файлов
        for data_file in tqdm(self._data_files, desc='Adding data'):
            model_tags = list()
            rules_tags = list()
            # открываем файл с тегами, полученными моделью
            with open(os.path.join(self._data_ml, data_file), 'r', encoding='utf-8') as data_f:
                reader = csv.DictReader(data_f)
                for row in reader:
                    model_tags.append(row['predicted_ml_tag'])
            with open(os.path.join(self._data_rules, data_file), 'r', encoding='utf-8') as data_f:
                rules_reader = csv.DictReader(data_f)
                # записываем результаты в новый файл
                with open(os.path.join(self._path_to_data_dir, 'combined', data_file), 'w', encoding='utf-8', newline='') as task:
                        fieldnames = ['token', 'tag', self.column_name]
                        writer = csv.DictWriter(task, fieldnames=fieldnames)
                        writer.writeheader()
                        for i, row in enumerate(rules_reader):
                            ml_tag = model_tags[i]     
                            rules_tag = row['predicted_rules_tag']      
                            gtags = ['B-DATE', 'I-DATE', 'B-DURATION', 'I-DURATION'] 
                            mtags = ['B-TIME', 'I-TIME', 'B-SET', 'I-SET']

                            if rules_tag != ml_tag:
                                if rules_tag in mtags:
                                    result_tag = rules_tag
                                elif ml_tag in mtags:
                                    result_tag = rules_tag
                                
                                elif (rules_tag == 'O') and (ml_tag.endswith('DATE') == True):
                                    result_tag = ml_tag
                                elif (rules_tag == 'O') and (ml_tag.endswith('DATE') == False):
                                    result_tag = rules_tag

                                elif (rules_tag.endswith('DATE') == True) and (ml_tag == 'O'):
                                    result_tag = ml_tag
                                elif (rules_tag.endswith('DATE') == True) and (ml_tag.endswith('DURATION') == True):
                                    result_tag = ml_tag
                                
                                elif (rules_tag.endswith('DURATION') == True) and (ml_tag.endswith('DATE') == True):
                                    result_tag = ml_tag
                                elif (rules_tag.endswith('DURATION') == True) and (ml_tag == 'O'):
                                    result_tag = rules_tag
                                            
            

                                writer.writerow({'token' : row['token'], 'tag' : row['tag'], self.column_name: result_tag})  
                            else:
                                writer.writerow({'token' : row['token'], 'tag' : row['tag'], self.column_name: ml_tag})